<a href="https://colab.research.google.com/github/H24-D/Gen-ai-lab/blob/main/program5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain-google-genai gradio

In [29]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
import gradio as gr
from getpass import getpass
import os
GOOGLE_API_KEY=getpass("Enter your google api key")
llm=ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.8,
    api_key=GOOGLE_API_KEY,
    max_tokens=512,
    timeout=30,
    max_retries=3
)
print("Gemini LLM is ready")


Enter your google api key··········
Gemini LLM is ready


In [30]:
def get_similar_words(seed):
    prompt = (
        f"Give me 5 English words that are semantically or creatively similar to '{seed}'. "
        f"Return the words as a comma-separated list without numbers or explanations."
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    return [word.strip() for word in response.content.split(',') if word.strip()]


In [31]:
def create_paragraph(seed, words):
    word_list = ', '.join(words)
    prompt = (
        f"Write a short, creative paragraph using the word '{seed}' and the following related words: "
        f"{word_list}. The paragraph should be imaginative and meaningful."
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    return response.content

In [32]:
def generate_paragraphs(seed_word):
    try:
        seed_word = seed_word.strip()
        if not seed_word:
            return "Please enter a valid seed word."

        similar_words = get_similar_words(seed_word)
        if len(similar_words) < 3:
            return "Could not find similar words. Try a different seed word."

        output_text = f"Seed Word: {seed_word}\nSimilar Words: {', '.join(similar_words)}\n\n"

        for i in range(1, 4):
            paragraph = create_paragraph(seed_word, similar_words) or f"(Variation {i}) Could not generate paragraph."
            output_text += f"--- Variation {i} ---\n{paragraph.strip()}\n\n"

        return output_text

    except Exception as e:
        return f"Error: {str(e)}"


In [33]:
gr.Interface(
    fn=generate_paragraphs,
    inputs=gr.Textbox(label="Enter a Seed Word"),
    outputs=gr.Markdown(label="Generated Paragraphs"),
    title="Creative Writer",
    description="Enter a seed word. This app will find similar words using Gemini and generate 3 creative paragraph variations.",
    theme="default",
).launch(debug=False)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9c4528efd86a1e6074.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
